In [1]:
import pandas as pd
import requests
from secrets_config import api_key_id, api_secret_key 
import jinja2 as j2 

Extract data

In [2]:
# retrieve data for tesla between 2020-01-01 to 2020-01-02
# auth example: https://alpaca.markets/docs/api-references/trading-api/

stock_ticker = "tsla"
base_url = f"https://data.alpaca.markets/v2/stocks/{stock_ticker}/trades"
start_time = "2020-01-01T00:00:00.00Z"
end_time = "2020-01-02T00:00:00.00Z"

response_data = []

params = {
    "start": start_time,
    "end": end_time
}

headers = {
    "APCA-API-KEY-ID": api_key_id,
    "APCA-API-SECRET-KEY": api_secret_key
}
response = requests.get(base_url, params=params, headers=headers)
if response.json().get("trades") is not None: 
    response_data.extend(response.json().get("trades"))

In [3]:
# read trade data into a dataframe
df = pd.json_normalize(data=response_data, max_level=0)

In [4]:
# read exchange codes data into a dataframe 
df_exchange_codes = pd.read_csv("data/exchange_codes.csv")

Load data

In [5]:
# import libraries for sql 
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float, JSON 
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secrets_config import db_user, db_password, db_server_name, db_database_name

In [6]:
# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)

engine = create_engine(connection_url)

In [7]:
# create meta object 
meta = MetaData()

# specify trade data table schema 
# see field definition here: https://alpaca.markets/docs/api-references/market-data-api/stock-pricing-data/historical/#trade 
trades_table = Table(
    "raw_trades", meta, 
    Column("i", Integer, primary_key=True),
    Column("t", String),
    Column("x", String),
    Column("p", Float),
    Column("s", Integer),
    Column("c", String),
    Column("z", String)
)

# specify exchange codes table schema 
exchange_codes_table = Table(
    "raw_exchange_codes", meta, 
    Column("exchange_code", String, primary_key=True), 
    Column("exchange_name", String)
)

# creates table if it does not exist 
meta.create_all(engine) 

In [8]:
# upsert data to trade table 
insert_statement = postgresql.insert(trades_table).values(df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['i'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['i']})
engine.execute(upsert_statement)

# upsert data to exchange codes table 
insert_statement = postgresql.insert(exchange_codes_table).values(df_exchange_codes.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['exchange_code'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['exchange_code']})
engine.execute(upsert_statement)

Transform

In [10]:
## for staging_trades
# create variable for target table 
target_table = "staging_trades"

# read sql contents into a variable 
with open(f"models/{target_table}.sql") as f: 
    raw_sql = f.read()

# parse sql using jinja 
parsed_sql = j2.Template(raw_sql).render(target_table = target_table)

# execute parsed sql 
engine.execute(parsed_sql)

In [11]:
## for staging_exchange_codes
# create variable for target table 
target_table = "staging_exchange_codes"

# read sql contents into a variable 
with open(f"models/{target_table}.sql") as f: 
    raw_sql = f.read()

# parse sql using jinja 
parsed_sql = j2.Template(raw_sql).render(target_table = target_table)

# execute parsed sql 
engine.execute(parsed_sql)

In [13]:
## for serving_trades
# create variable for target table 
target_table = "serving_trades"

# read sql contents into a variable 
with open(f"models/{target_table}.sql") as f: 
    raw_sql = f.read()

# parse sql using jinja 
parsed_sql = j2.Template(raw_sql).render(target_table = target_table)

# execute parsed sql 
engine.execute(parsed_sql)